# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load 
file_to_load = "./output_cities.csv"
outputcities = pd.read_csv(file_to_load)
outputcities

,City ID,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloud Cover,Wind Speed,Country,Date
0,0,avarua,-21.21,-159.78,299.15,69,96,6.20,CK,1591757261
1,1,te anau,-45.42,167.72,284.21,63,29,4.30,NZ,1591757260
2,2,acara,-1.96,-48.20,295.64,98,100,0.83,BR,1591757506
3,3,hobart,-42.88,147.33,286.48,47,75,5.70,AU,1591757366
4,4,jamestown,42.10,-79.24,298.15,53,1,2.10,US,1591757506
...,...,...,...,...,...,...,...,...,...,...
545,545,yurga,55.72,84.89,285.89,83,100,2.40,RU,1591757562
546,546,upata,8.01,-62.40,300.37,82,63,0.45,VE,1591757562
547,547,batagay-alyta,67.80,130.41,278.24,67,100,3.78,RU,1591757401
548,548,koslan,63.46,48.90,294.12,69,100,4.44,RU,1591757562


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
places = outputcities[['Latitude', "Longitude"]]
fig = gmaps.figure(zoom_level=2, center = (0,0))
heat_layer = gmaps.heatmap_layer(places, weights = outputcities["Humidity"], dissipating = False, max_intensity = 250, point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Max Temp Converted from Kelvin
outputcitiesdf = outputcities.loc[(outputcities["Maximum Temperature"] > 294) & 
                        (outputcities["Maximum Temperature"] < 300) & (outputcities["Cloud Cover"] == 0)]
idealcitydf = outputcitiesdf.dropna(how='any')
idealcitydf.reset_index(inplace=True)
del idealcitydf['index']
idealcitydf.head()
outputcitiesdf

,City ID,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloud Cover,Wind Speed,Country,Date
57,57,carnarvon,-24.87,113.63,296.55,27,0,0.52,AU,1591757121
72,72,sao joao da barra,-21.64,-41.05,295.20,89,0,4.59,BR,1591757376
81,81,buraydah,26.33,43.98,299.15,22,0,2.10,SA,1591757372
83,83,east london,-33.02,27.91,298.21,28,0,6.98,ZA,1591757261
106,106,albany,42.60,-73.97,294.26,56,0,1.51,US,1591757516
126,126,aswan,24.09,32.91,299.72,17,0,3.65,EG,1591757340
147,147,itarema,-2.92,-39.92,298.04,85,0,3.50,BR,1591757278
177,177,hun,29.13,15.95,297.17,46,0,3.19,LY,1591757383
189,189,delijan,33.99,50.68,297.08,19,0,2.01,IR,1591757524
190,190,sao miguel do araguaia,-13.28,-50.16,295.44,63,0,1.36,BR,1591757524


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = idealcitydf.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City ID,City,Latitude,Longitude,Maximum Temperature,Humidity,Cloud Cover,Wind Speed,Country,Date,Hotel Name
0,57,carnarvon,-24.87,113.63,296.55,27,0,0.52,AU,1591757121,
1,72,sao joao da barra,-21.64,-41.05,295.20,89,0,4.59,BR,1591757376,
2,81,buraydah,26.33,43.98,299.15,22,0,2.10,SA,1591757372,
3,83,east london,-33.02,27.91,298.21,28,0,6.98,ZA,1591757261,
4,106,albany,42.60,-73.97,294.26,56,0,1.51,US,1591757516,


In [6]:
base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params= {"radius": 5000, "type": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params["location"]=f'{lat},{lng}'
    city= row["City"]
    print(f"Finding hotel in {city}")
    response=requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except(KeyError, IndexError):
            print("Missing field/result...skipping")

Finding hotel in carnarvon
Finding hotel in sao joao da barra
Finding hotel in buraydah
Finding hotel in east london
Finding hotel in albany
Missing field/result...skipping
Finding hotel in aswan
Finding hotel in itarema
Finding hotel in hun
Finding hotel in delijan
Finding hotel in sao miguel do araguaia
Finding hotel in erenhot
Finding hotel in zuwarah
Finding hotel in guerrero negro
Finding hotel in verkhnyaya sinyachikha
Finding hotel in hobyo
Finding hotel in bogdanovich
Finding hotel in svobodnyy
Finding hotel in floriano
Finding hotel in nhamunda
Finding hotel in caravelas
Finding hotel in safaga
Finding hotel in asyut


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markerlayer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markerlayer)
# Display figure

fig

Figure(layout=FigureLayout(height='420px'))